# Ray Tune - Basic Tune Concepts and Steps: Warming Up with Ray Tune Hyperparameter Tuning

© 2019-2022, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademyLogo.png)

This lesson introduces Ray tune's key concepts using a trivial examples. This example is derived from [Ray Tune basic example](https://docs.ray.io/en/latest/tune/examples/tune_basic_example.html). Basically, there are three basic steps or Ray Tune pattern for you as a newcomer to get started with using Ray Tune.

 1. Setup your config space and define your trainable and objective function
 2. Use tune to execute your training, supplying the appropriate arguments including: search space, [search algorithms](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#blendsearch) or [trial schedulers](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers)
 3. Examine analyse the results
 
 ![](https://docs.ray.io/en/latest/_images/tune-workflow.png)


See also the [Hyperparameter Tuning References](References-Hyperparameter-Tuning.ipynb) notebook and the [Tune documentation](http://tune.io), in particular, the [API reference](https://docs.ray.io/en/latest/tune/api_docs/overview.html). 


### Install Ray Tune
[Ray Tune](https://docs.ray.io/en/master/installation.html#official-releases) requires a separate install. 

In [1]:
!pip install 'ray[tune]'

### 1. Setup training using Trainable APIs

In [2]:
import time

import ray
from ray import tune

Let's define our objective function

In [3]:
def evaluation_fn(step, width, height):
    time.sleep(0.1)
    return (0.1 + width * step / 100)**(-1) + height * 0.1

Next, we define a Trainable used by Tune using Tune's [Functional API](https://docs.ray.io/en/latest/tune/api_docs/trainable.html#function-api)

In [4]:
def easy_objective(config):
    # fetch our Hyperparameters sent as arguments
    width, height = config["width"], config["height"]
    # Iterate over number of steps
    for step in range(config["steps"]):
        # Iterative training function - can be any arbitrary training procedure
        # Here our objective function is the evaluation_fn
        intermediate_score = evaluation_fn(step, width, height)
        # Feed the score back back to Tune.
        tune.report(iterations=step, mean_loss=intermediate_score)

In [5]:
ray.init(ignore_reinit_error=True)

2022-02-21 11:37:49,690	INFO services.py:1376 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': '/tmp/ray/session_2022-02-21_11-37-47_110420_12908/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-02-21_11-37-47_110420_12908/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-02-21_11-37-47_110420_12908',
 'metrics_export_port': 58043,
 'gcs_address': '127.0.0.1:54505',
 'node_id': '991d94246fa8dd503fbe7c3cb29784e7f030b5f3581d4c82af5bfba5'}

### Step 2. Use tune API to execute tuning

This will do a grid search over the `activation` parameter. This means
that each of the two values (`relu` and `tanh`) will be sampled once
for each sample (`num_samples`). We end up with 2 * N = 2N samples, where is `num_samples`
The `width` and `height` parameters are sampled randomly.
`steps` is a constant parameter.

The `tune.run(..)` API returns a large [analysis](https://docs.ray.io/en/latest/tune/api_docs/analysis.html#analysis-tune-analysis) object. 

In [6]:
analysis = tune.run(
    easy_objective,
    metric="mean_loss",
    mode="min",
    num_samples=5,
    # Define our hypyerparameter search space
    config={
        "steps": 5,
        "width": tune.uniform(0, 20),
        "height": tune.uniform(-100, 100),
        "activation": tune.grid_search(["relu", "tanh"]),
    },
    verbose=1
)

2022-02-21 11:38:02,698	INFO tune.py:636 -- Total run time: 3.39 seconds (2.20 seconds for the tuning loop).


### Step 3. Analyse the results

In [7]:
print("Best hyperparameters found were: ", analysis.best_config)

Best hyperparameters found were:  {'steps': 5, 'width': 8.896576435350273, 'height': -83.21313463274562, 'activation': 'relu'}


Alternatively, you can examine a Pandas dataframe

In [8]:
analysis.results_df.head(5)

/usr/local/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/ray/tune/analysis/experiment_analysis.py:281: UserWarning: Dataframes will use '/' instead of '.' to delimit nested result keys in future versions of Ray. For forward compatibility, set the environment variable TUNE_RESULT_DELIM='/'
  "Dataframes will use '/' instead of '.' to delimit "


,iterations,mean_loss,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,neg_mean_loss,experiment_id,date,...,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,experiment_tag,config.steps,config.width,config.height,config.activation
trial_id,,,,,,,,,,,,,,,,,,,,,
c6778_00000,4,7.710453,0.103652,True,None,None,5,-7.710453,5ff395ed47dd472380085aad4c13f715,2022-02-21_11-38-01,...,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,0.530236,0,5,"0_activation=relu,height=63.055,width=15.294",5,15.293927,63.054790,relu
c6778_00001,4,2.826230,0.104425,True,None,None,5,-2.826230,eb30c25ae4c641ba9406fc0afab3fbfe,2022-02-21_11-38-02,...,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,0.531171,0,5,"1_activation=tanh,height=15.118,width=16.52",5,16.519595,15.117961,tanh
c6778_00002,4,5.776167,0.103554,True,None,None,5,-5.776167,f6997093c62b47d199fca9f4d0c49a2f,2022-02-21_11-38-02,...,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,0.532462,0,5,"2_activation=relu,height=35.183,width=8.5723",5,8.572295,35.182791,relu
c6778_00003,4,6.995739,0.105229,True,None,None,5,-6.995739,5234fac9e9ef4ead80dcebd45dda9c3a,2022-02-21_11-38-02,...,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,0.532118,0,5,"3_activation=tanh,height=39.461,width=5.6977",5,5.697668,39.460914,tanh
c6778_00004,4,-6.127672,0.104826,True,None,None,5,6.127672,0f5c3a1bfa354c2bbab5e8332c99ed37,2022-02-21_11-38-02,...,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,0.538095,0,5,"4_activation=relu,height=-83.213,width=8.8966",5,8.896576,-83.213135,relu


In [9]:
analysis.trials

[easy_objective_c6778_00000,
 easy_objective_c6778_00001,
 easy_objective_c6778_00002,
 easy_objective_c6778_00003,
 easy_objective_c6778_00004,
 easy_objective_c6778_00005,
 easy_objective_c6778_00006,
 easy_objective_c6778_00007,
 easy_objective_c6778_00008,
 easy_objective_c6778_00009]

In [10]:
analysis.dataframe(metric="mean_loss", mode="min").head(5)

,iterations,mean_loss,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,neg_mean_loss,trial_id,experiment_id,...,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,config/activation,config/height,config/steps,config/width,logdir
0,4,7.710453,0.103652,False,NaN,NaN,5,-7.710453,c6778_00000,5ff395ed47dd472380085aad4c13f715,...,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,0.530236,0,5,relu,63.054790,5,15.293927,/Users/jules/ray_results/easy_objective_2022-0...
1,4,2.826230,0.104425,False,NaN,NaN,5,-2.826230,c6778_00001,eb30c25ae4c641ba9406fc0afab3fbfe,...,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,0.531171,0,5,tanh,15.117961,5,16.519595,/Users/jules/ray_results/easy_objective_2022-0...
2,4,5.776167,0.103554,False,NaN,NaN,5,-5.776167,c6778_00002,f6997093c62b47d199fca9f4d0c49a2f,...,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,0.532462,0,5,relu,35.182791,5,8.572295,/Users/jules/ray_results/easy_objective_2022-0...
3,4,6.995739,0.105229,False,NaN,NaN,5,-6.995739,c6778_00003,5234fac9e9ef4ead80dcebd45dda9c3a,...,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,0.532118,0,5,tanh,39.460914,5,5.697668,/Users/jules/ray_results/easy_objective_2022-0...
4,4,-6.127672,0.104826,False,NaN,NaN,5,6.127672,c6778_00004,0f5c3a1bfa354c2bbab5e8332c99ed37,...,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,0.538095,0,5,relu,-83.213135,5,8.896576,/Users/jules/ray_results/easy_objective_2022-0...


In [11]:
ray.shutdown()